# NutriLearn AI - Food Classification Model Training

This notebook trains a deep learning model for food image classification using transfer learning.

**Features:**
- Transfer learning with MobileNetV2 or EfficientNet-B0
- Food-101 dataset (101 food classes)
- MLflow experiment tracking
- Automatic checkpointing and model saving
- GPU acceleration (if available)

**Author:** NutriLearn AI Team  
**Purpose:** B.Tech Final Year AI/ML Project

## 1. Setup Environment

Install required packages and check GPU availability.

In [ ]:
# Install required packages
!pip install torch torchvision mlflow tqdm matplotlib seaborn scikit-learn -q

print("\n" + "="*80)
print("Installation Complete!")
print("="*80)

In [ ]:
# Check GPU availability
import torch

print("\n" + "="*80)
print("GPU Information")
print("="*80)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"✓ GPU Available: {gpu_name}")
    print(f"✓ GPU Memory: {gpu_memory:.2f} GB")
    print(f"✓ CUDA Version: {torch.version.cuda}")
else:
    print("⚠ No GPU available - training will use CPU")
    print("⚠ Training on CPU will be significantly slower")
    print("⚠ Consider using a GPU runtime: Runtime > Change runtime type > GPU")

print("="*80)

## 2. Mount Google Drive

Mount Google Drive for persistent storage of datasets and trained models.

In [ ]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

print("\n" + "="*80)
print("Google Drive mounted successfully!")
print("="*80)
print("\nYou can now save models and datasets to:")
print("  /content/drive/MyDrive/nutrilearn-ai/")
print("="*80)

## 3. Download Food-101 Dataset

Download and extract the Food-101 dataset (5GB compressed, ~7GB extracted).

**Note:** This will take 10-15 minutes depending on your connection speed.

In [ ]:
import os

# Create data directory
data_dir = "/content/food-101-data"
os.makedirs(data_dir, exist_ok=True)

print("="*80)
print("Downloading Food-101 Dataset")
print("="*80)
print("Dataset: Food-101 (101 food categories, 101,000 images)")
print("Size: ~5GB compressed, ~7GB extracted")
print("Source: ETH Zurich")
print("="*80)
print("\nThis will take 10-15 minutes...\n")

# The dataset will be automatically downloaded by torchvision.datasets.Food101
# on first run of the training script
print("✓ Data directory created: {}".format(data_dir))
print("✓ Dataset will be auto-downloaded during training")
print("="*80)

## 4. Upload Training Script

Upload the `train_model.py` script from your local machine or clone from repository.

In [ ]:
# Option 1: Upload train_model.py manually using Colab's file upload
from google.colab import files

print("="*80)
print("Upload train_model.py")
print("="*80)
print("Please upload the train_model.py file from your local machine.")
print("="*80)

uploaded = files.upload()

if 'train_model.py' in uploaded:
    print("\n✓ train_model.py uploaded successfully!")
else:
    print("\n⚠ train_model.py not found. Please upload it.")

In [ ]:
# Option 2: Clone from GitHub repository (if available)
# Uncomment and modify the following lines if you have a GitHub repo:

# !git clone https://github.com/your-username/nutrilearn-ai.git
# %cd nutrilearn-ai/backend
# !ls -la train_model.py

## 5. Train the Model

Run the training script with your desired hyperparameters.

**Training Configuration:**
- Model: MobileNetV2 (lightweight, fast)
- Epochs: 15 (adjust based on time/resources)
- Batch Size: 32 (reduce to 16 if out of memory)
- Learning Rate: 0.001
- Early Stopping: Enabled (patience=5)

**Expected Training Time:**
- With GPU (T4): ~45-60 minutes for 15 epochs
- With CPU: ~8-10 hours (not recommended)

In [ ]:
# Train with default settings (MobileNetV2, 15 epochs)
!python train_model.py \
    --data-dir {data_dir} \
    --epochs 15 \
    --batch-size 32 \
    --learning-rate 0.001 \
    --model-name mobilenet_v2 \
    --early-stopping-patience 5 \
    --experiment-name "colab_food_classification"

### Alternative: Train with EfficientNet-B0 (Better Accuracy)

Uncomment the cell below to train with EfficientNet-B0 instead.

In [ ]:
# Train with EfficientNet-B0 (better accuracy, slightly slower)
# !python train_model.py \
#     --data-dir {data_dir} \
#     --epochs 20 \
#     --batch-size 32 \
#     --learning-rate 0.001 \
#     --model-name efficientnet_b0 \
#     --early-stopping-patience 5 \
#     --experiment-name "colab_food_classification_efficientnet"

### Resume Training from Checkpoint

If training was interrupted, you can resume from the last checkpoint.

In [ ]:
# Resume training from checkpoint
# !python train_model.py \
#     --data-dir {data_dir} \
#     --epochs 20 \
#     --checkpoint-path ./checkpoints/checkpoint_mobilenet_v2_YYYYMMDD_HHMMSS.pth

## 6. View Training Results

Display training curves and sample predictions.

In [ ]:
# Display training curves
from IPython.display import Image, display
import os

print("="*80)
print("Training Results")
print("="*80)

if os.path.exists('training_curves.png'):
    print("\nTraining Curves:")
    display(Image('training_curves.png'))
else:
    print("⚠ training_curves.png not found")

if os.path.exists('confusion_matrix.png'):
    print("\nConfusion Matrix:")
    display(Image('confusion_matrix.png'))
else:
    print("⚠ confusion_matrix.png not found")

if os.path.exists('sample_predictions.png'):
    print("\nSample Predictions:")
    display(Image('sample_predictions.png'))
else:
    print("⚠ sample_predictions.png not found")

## 7. Download Trained Model

Download the trained model files to your local machine.

In [ ]:
# Find the latest model directory
import os
import glob
from google.colab import files

model_dirs = glob.glob('ml-models/nutrilearn_model_*')

if model_dirs:
    latest_model_dir = max(model_dirs, key=os.path.getctime)
    
    print("="*80)
    print("Downloading Trained Model Files")
    print("="*80)
    print(f"Model directory: {latest_model_dir}")
    print("="*80)
    
    # Download model files
    model_files = [
        'food_model_v1.pth',
        'class_to_idx.json',
        'model_config.json',
        'evaluation_results.json'
    ]
    
    for filename in model_files:
        filepath = os.path.join(latest_model_dir, filename)
        if os.path.exists(filepath):
            print(f"\nDownloading {filename}...")
            files.download(filepath)
            print(f"✓ {filename} downloaded")
        else:
            print(f"⚠ {filename} not found")
    
    print("\n" + "="*80)
    print("Download Complete!")
    print("="*80)
else:
    print("⚠ No trained models found. Please run training first.")

## 8. Save to Google Drive (Optional)

Copy trained models to Google Drive for persistent storage.

In [ ]:
# Copy models to Google Drive
import shutil
import os

drive_save_dir = '/content/drive/MyDrive/nutrilearn-ai/models'
os.makedirs(drive_save_dir, exist_ok=True)

model_dirs = glob.glob('ml-models/nutrilearn_model_*')

if model_dirs:
    latest_model_dir = max(model_dirs, key=os.path.getctime)
    
    print("="*80)
    print("Saving to Google Drive")
    print("="*80)
    print(f"Source: {latest_model_dir}")
    print(f"Destination: {drive_save_dir}")
    print("="*80)
    
    # Copy entire model directory
    dest_dir = os.path.join(drive_save_dir, os.path.basename(latest_model_dir))
    shutil.copytree(latest_model_dir, dest_dir, dirs_exist_ok=True)
    
    print(f"\n✓ Models saved to Google Drive: {dest_dir}")
    print("\nYour models are now safely stored in Google Drive!")
    print("="*80)
else:
    print("⚠ No trained models found to save.")

## 9. Test Inference (Optional)

Test the trained model with a sample image.

In [ ]:
# Quick inference test
import torch
import json
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Load model
model_dirs = glob.glob('ml-models/nutrilearn_model_*')
if model_dirs:
    latest_model_dir = max(model_dirs, key=os.path.getctime)
    
    # Load model
    model_path = os.path.join(latest_model_dir, 'food_model_v1.pth')
    class_mapping_path = os.path.join(latest_model_dir, 'class_to_idx.json')
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torch.load(model_path, map_location=device)
    model.eval()
    
    # Load class mapping
    with open(class_mapping_path, 'r') as f:
        class_to_idx = json.load(f)
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    print("="*80)
    print("Model Loaded Successfully!")
    print("="*80)
    print(f"Model: {model_path}")
    print(f"Classes: {len(class_to_idx)}")
    print(f"Device: {device}")
    print("="*80)
    print("\nReady for inference!")
    print("Upload a food image to test the model.")
else:
    print("⚠ No trained models found. Please run training first.")

## 10. Summary

**What you've accomplished:**
1. ✓ Set up Google Colab environment with GPU
2. ✓ Downloaded Food-101 dataset
3. ✓ Trained a food classification model
4. ✓ Evaluated model performance
5. ✓ Downloaded trained model files
6. ✓ Saved models to Google Drive

**Next Steps:**
1. Integrate the trained model into your FastAPI backend
2. Test the model with real food images
3. Deploy the application
4. Monitor model performance in production

**Model Files:**
- `food_model_v1.pth` - Trained model weights
- `class_to_idx.json` - Class name to index mapping
- `model_config.json` - Model architecture configuration
- `evaluation_results.json` - Performance metrics

**Resources:**
- [PyTorch Documentation](https://pytorch.org/docs/)
- [Food-101 Dataset](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/)
- [Transfer Learning Guide](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)